In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
resumen_historico = pd.read_csv('./datos_dashboard/datos_v2 - resumen historico.csv')
resumen_historico.Fecha = pd.to_datetime(resumen_historico.ID.str[0:10], format="%d-%m-%Y")
resumen_historico

,ID,Fecha,Origen backlog,Tabla,Reglas ID DATUM OK (IMPLANTABLES),Reglas DEV OK (WIP FINAL),Reglas Blocked,Pendientes DEV,UUAA
0,02-10-2023DOM - C360 GDHt_b0hn_sust_group_mark,2023-10-02,DOM - C360 GDH,t_b0hn_sust_group_mark,26,26,0,0,B0HN
1,02-10-2023DOM - C360 GDHt_b0hn_sust_groups_fdbk,2023-10-02,DOM - C360 GDH,t_b0hn_sust_groups_fdbk,15,15,0,0,B0HN
2,02-10-2023DOM - C360 GDHt_b0hn_sust_groups_fdb...,2023-10-02,DOM - C360 GDH,t_b0hn_sust_groups_fdbk_detail,15,15,0,0,B0HN
3,02-10-2023DOM - C360 GDHt_cgj7_customer_carbon...,2023-10-02,DOM - C360 GDH,t_cgj7_customer_carbon_footprint,2,2,0,0,CGJ7
4,02-10-2023DOM - C360 GDHt_cgj7_sust_customer_mark,2023-10-02,DOM - C360 GDH,t_cgj7_sust_customer_mark,24,24,0,0,CGJ7
...,...,...,...,...,...,...,...,...,...
26672,24-02-2025MVP - Adaptacionest_tdi3_cust_promo_...,2025-02-24,MVP - Adaptaciones,t_tdi3_cust_promo_adhesion_dp,11,11,0,0,TDI3
26673,24-02-2025MVP - Adaptacionest_tdi3_cust_promot...,2025-02-24,MVP - Adaptaciones,t_tdi3_cust_promotions,7,7,0,0,TDI3
26674,24-02-2025MVP - Adaptacionest_tdi3_tracking_code,2025-02-24,MVP - Adaptaciones,t_tdi3_tracking_code,13,13,0,0,TDI3
26675,24-02-2025MVP - Adaptacionest_tlhr_complaints_...,2025-02-24,MVP - Adaptaciones,t_tlhr_complaints_claims,14,14,0,0,TLHR


In [3]:
fechas = pd.DataFrame( {'Fecha':resumen_historico.Fecha.unique()})
fechas = fechas.sort_values('Fecha',ascending=True)
fechas['Fecha_past'] = [fechas.Fecha[0] - pd.Timedelta(days=7)] + list(fechas.Fecha[0:-1])
fechas.head()

,Fecha,Fecha_past
0,2023-10-02,2023-09-25
1,2023-10-09,2023-10-02
2,2023-10-16,2023-10-09
3,2023-10-23,2023-10-16
4,2023-10-30,2023-10-23


In [4]:
#TODO: Realizar solo para 'Reglas ID DATUM OK (IMPLANTABLES)','Reglas DEV OK (WIP FINAL)'
# sobre todo cuando looker o google sheets pete por tamaño, se reducirá a la mitad el número de filas con esta medida

long_diff = resumen_historico.melt(id_vars='ID',value_vars=[ 'Reglas ID DATUM OK (IMPLANTABLES)','Reglas DEV OK (WIP FINAL)', 'Reglas Blocked', 'Pendientes DEV'],   var_name='COL', value_name='VAL')[['COL','ID','VAL']]
long_diff.insert(loc=0, column='ID_COL_Copy', value=long_diff.ID + long_diff.COL)
long_diff = long_diff.merge(resumen_historico[['ID','Fecha','Origen backlog','Tabla','UUAA']], on = 'ID', how='left')
long_diff = long_diff.merge(fechas, on = 'Fecha', how='left')
long_diff['ID_COL_past'] = long_diff.Fecha_past.dt.strftime('%d-%m-%Y') + long_diff['Origen backlog'] + long_diff.Tabla + long_diff.COL
long_diff['VAL_past'] = long_diff.merge(long_diff[['ID_COL_Copy','VAL']], left_on = 'ID_COL_past', right_on = 'ID_COL_Copy', how='left')['VAL_y'].fillna(0)
long_diff['VAL_diff'] = long_diff.VAL - long_diff.VAL_past
print(long_diff.columns)
long_diff

Index(['ID_COL_Copy', 'COL', 'ID', 'VAL', 'Fecha', 'Origen backlog', 'Tabla',
       'UUAA', 'Fecha_past', 'ID_COL_past', 'VAL_past', 'VAL_diff'],
      dtype='object')


,ID_COL_Copy,COL,ID,VAL,Fecha,Origen backlog,Tabla,UUAA,Fecha_past,ID_COL_past,VAL_past,VAL_diff
0,02-10-2023DOM - C360 GDHt_b0hn_sust_group_mark...,Reglas ID DATUM OK (IMPLANTABLES),02-10-2023DOM - C360 GDHt_b0hn_sust_group_mark,26,2023-10-02,DOM - C360 GDH,t_b0hn_sust_group_mark,B0HN,2023-09-25,25-09-2023DOM - C360 GDHt_b0hn_sust_group_mark...,0.0,26.0
1,02-10-2023DOM - C360 GDHt_b0hn_sust_groups_fdb...,Reglas ID DATUM OK (IMPLANTABLES),02-10-2023DOM - C360 GDHt_b0hn_sust_groups_fdbk,15,2023-10-02,DOM - C360 GDH,t_b0hn_sust_groups_fdbk,B0HN,2023-09-25,25-09-2023DOM - C360 GDHt_b0hn_sust_groups_fdb...,0.0,15.0
2,02-10-2023DOM - C360 GDHt_b0hn_sust_groups_fdb...,Reglas ID DATUM OK (IMPLANTABLES),02-10-2023DOM - C360 GDHt_b0hn_sust_groups_fdb...,15,2023-10-02,DOM - C360 GDH,t_b0hn_sust_groups_fdbk_detail,B0HN,2023-09-25,25-09-2023DOM - C360 GDHt_b0hn_sust_groups_fdb...,0.0,15.0
3,02-10-2023DOM - C360 GDHt_cgj7_customer_carbon...,Reglas ID DATUM OK (IMPLANTABLES),02-10-2023DOM - C360 GDHt_cgj7_customer_carbon...,2,2023-10-02,DOM - C360 GDH,t_cgj7_customer_carbon_footprint,CGJ7,2023-09-25,25-09-2023DOM - C360 GDHt_cgj7_customer_carbon...,0.0,2.0
4,02-10-2023DOM - C360 GDHt_cgj7_sust_customer_m...,Reglas ID DATUM OK (IMPLANTABLES),02-10-2023DOM - C360 GDHt_cgj7_sust_customer_mark,24,2023-10-02,DOM - C360 GDH,t_cgj7_sust_customer_mark,CGJ7,2023-09-25,25-09-2023DOM - C360 GDHt_cgj7_sust_customer_m...,0.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...
106703,24-02-2025MVP - Adaptacionest_tdi3_cust_promo_...,Pendientes DEV,24-02-2025MVP - Adaptacionest_tdi3_cust_promo_...,0,2025-02-24,MVP - Adaptaciones,t_tdi3_cust_promo_adhesion_dp,TDI3,2025-02-17,17-02-2025MVP - Adaptacionest_tdi3_cust_promo_...,0.0,0.0
106704,24-02-2025MVP - Adaptacionest_tdi3_cust_promot...,Pendientes DEV,24-02-2025MVP - Adaptacionest_tdi3_cust_promot...,0,2025-02-24,MVP - Adaptaciones,t_tdi3_cust_promotions,TDI3,2025-02-17,17-02-2025MVP - Adaptacionest_tdi3_cust_promot...,0.0,0.0
106705,24-02-2025MVP - Adaptacionest_tdi3_tracking_co...,Pendientes DEV,24-02-2025MVP - Adaptacionest_tdi3_tracking_code,0,2025-02-24,MVP - Adaptaciones,t_tdi3_tracking_code,TDI3,2025-02-17,17-02-2025MVP - Adaptacionest_tdi3_tracking_co...,0.0,0.0
106706,24-02-2025MVP - Adaptacionest_tlhr_complaints_...,Pendientes DEV,24-02-2025MVP - Adaptacionest_tlhr_complaints_...,0,2025-02-24,MVP - Adaptaciones,t_tlhr_complaints_claims,TLHR,2025-02-17,17-02-2025MVP - Adaptacionest_tlhr_complaints_...,0.0,0.0


In [5]:
long_diff_resumen = long_diff[['Fecha', 'COL', 'Origen backlog', 'Tabla', 'UUAA', 'VAL_diff']]
long_diff_resumen['VAL_diff_pos'] = np.where(long_diff_resumen.VAL_diff<0,0,long_diff_resumen.VAL_diff)
long_diff_resumen.Fecha = long_diff_resumen.Fecha.dt.strftime('%d/%m/%Y')
long_diff_resumen.to_csv('./datos_dashboard/long_diff_resumen.csv',index=False)
long_diff_resumen.reset_index(drop=True)

,Fecha,COL,Origen backlog,Tabla,UUAA,VAL_diff,VAL_diff_pos
0,02/10/2023,Reglas ID DATUM OK (IMPLANTABLES),DOM - C360 GDH,t_b0hn_sust_group_mark,B0HN,26.0,26.0
1,02/10/2023,Reglas ID DATUM OK (IMPLANTABLES),DOM - C360 GDH,t_b0hn_sust_groups_fdbk,B0HN,15.0,15.0
2,02/10/2023,Reglas ID DATUM OK (IMPLANTABLES),DOM - C360 GDH,t_b0hn_sust_groups_fdbk_detail,B0HN,15.0,15.0
3,02/10/2023,Reglas ID DATUM OK (IMPLANTABLES),DOM - C360 GDH,t_cgj7_customer_carbon_footprint,CGJ7,2.0,2.0
4,02/10/2023,Reglas ID DATUM OK (IMPLANTABLES),DOM - C360 GDH,t_cgj7_sust_customer_mark,CGJ7,24.0,24.0
...,...,...,...,...,...,...,...
106703,24/02/2025,Pendientes DEV,MVP - Adaptaciones,t_tdi3_cust_promo_adhesion_dp,TDI3,0.0,0.0
106704,24/02/2025,Pendientes DEV,MVP - Adaptaciones,t_tdi3_cust_promotions,TDI3,0.0,0.0
106705,24/02/2025,Pendientes DEV,MVP - Adaptaciones,t_tdi3_tracking_code,TDI3,0.0,0.0
106706,24/02/2025,Pendientes DEV,MVP - Adaptaciones,t_tlhr_complaints_claims,TLHR,0.0,0.0


Reglas finalizadas para última fecha

In [6]:
fecha_max = pd.to_datetime(long_diff_resumen.Fecha.iloc[-1], format='%d/%m/%Y').strftime('%d/%m/%Y')
long_diff_resumen.query('(Fecha == @fecha_max) & (VAL_diff_pos>0) & (COL == "Reglas DEV OK (WIP FINAL)")', engine='python')

,Fecha,COL,Origen backlog,Tabla,UUAA,VAL_diff,VAL_diff_pos
53070,24/02/2025,Reglas DEV OK (WIP FINAL),DOM - C360 GDH,t_rzua_operations_cust_scope_cib,RZUA,2.0,2.0
53075,24/02/2025,Reglas DEV OK (WIP FINAL),DOM - C360 GDH,t_rzua_sust_operations,RZUA,2.0,2.0
53076,24/02/2025,Reglas DEV OK (WIP FINAL),DOM - C360 GDH,t_rzua_sust_operations_cib,RZUA,2.0,2.0
53083,24/02/2025,Reglas DEV OK (WIP FINAL),DOM - C360 GDH,t_tdi3_cust_promo_adhesion_dp,TDI3,1.0,1.0
53294,24/02/2025,Reglas DEV OK (WIP FINAL),MVP - Adaptaciones,t_l9sv_digital_marketing_costs,L9SV,15.0,15.0
53331,24/02/2025,Reglas DEV OK (WIP FINAL),MVP - Adaptaciones,t_re4m_virtual_assistant_message,RE4M,19.0,19.0


In [7]:
fecha_max

'24/02/2025'

Valores positivos para ultima fecha

In [8]:
fecha_max = pd.to_datetime(long_diff_resumen.Fecha.iloc[-1], format='%d/%m/%Y').strftime('%d/%m/%Y')
long_diff_resumen.query('(Fecha == @fecha_max) & (VAL_diff_pos>0)', engine='python')

,Fecha,COL,Origen backlog,Tabla,UUAA,VAL_diff,VAL_diff_pos
26617,24/02/2025,Reglas ID DATUM OK (IMPLANTABLES),MVP - Adaptaciones,t_l9sv_digital_marketing_costs,L9SV,15.0,15.0
26654,24/02/2025,Reglas ID DATUM OK (IMPLANTABLES),MVP - Adaptaciones,t_re4m_virtual_assistant_message,RE4M,19.0,19.0
53070,24/02/2025,Reglas DEV OK (WIP FINAL),DOM - C360 GDH,t_rzua_operations_cust_scope_cib,RZUA,2.0,2.0
53075,24/02/2025,Reglas DEV OK (WIP FINAL),DOM - C360 GDH,t_rzua_sust_operations,RZUA,2.0,2.0
53076,24/02/2025,Reglas DEV OK (WIP FINAL),DOM - C360 GDH,t_rzua_sust_operations_cib,RZUA,2.0,2.0
53083,24/02/2025,Reglas DEV OK (WIP FINAL),DOM - C360 GDH,t_tdi3_cust_promo_adhesion_dp,TDI3,1.0,1.0
53294,24/02/2025,Reglas DEV OK (WIP FINAL),MVP - Adaptaciones,t_l9sv_digital_marketing_costs,L9SV,15.0,15.0
53331,24/02/2025,Reglas DEV OK (WIP FINAL),MVP - Adaptaciones,t_re4m_virtual_assistant_message,RE4M,19.0,19.0
